In [37]:
import rust_machine_learning
import pandas as pd
from syed_ml_lib import *

In [38]:
# Get the csv data
df = pd.read_csv("../data/iris.data")
# df = pd.read_csv("../data/kmeans-dataset.csv")
# df = pd.read_csv("../data/winequality-red.csv")
# df = pd.read_csv("../data/digit-recognizer/train.csv")
# df = df.sample(n=10000)
df  

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [39]:
to_encode = [4] # We won't encode the label column. The output layer will use 0-n (assuming classifications are ordered starting at 0), to determine error.
encoded, feature_to_encoded_cols = one_hot_encode(df, to_encode)
encoded

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),is_ class_Iris-setosa,is_ class_Iris-versicolor,is_ class_Iris-virginica
0,5.1,3.5,1.4,0.2,1.0,0.0,0.0
1,4.9,3.0,1.4,0.2,1.0,0.0,0.0
2,4.7,3.2,1.3,0.2,1.0,0.0,0.0
3,4.6,3.1,1.5,0.2,1.0,0.0,0.0
4,5.0,3.6,1.4,0.2,1.0,0.0,0.0
...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0.0,0.0,1.0
146,6.3,2.5,5.0,1.9,0.0,0.0,1.0
147,6.5,3.0,5.2,2.0,0.0,0.0,1.0
148,6.2,3.4,5.4,2.3,0.0,0.0,1.0


In [40]:
# Remove unnecessary data or NAN
to_remove = []
encoded = encoded.drop(encoded.columns[to_remove], axis=1)
encoded

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),is_ class_Iris-setosa,is_ class_Iris-versicolor,is_ class_Iris-virginica
0,5.1,3.5,1.4,0.2,1.0,0.0,0.0
1,4.9,3.0,1.4,0.2,1.0,0.0,0.0
2,4.7,3.2,1.3,0.2,1.0,0.0,0.0
3,4.6,3.1,1.5,0.2,1.0,0.0,0.0
4,5.0,3.6,1.4,0.2,1.0,0.0,0.0
...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0.0,0.0,1.0
146,6.3,2.5,5.0,1.9,0.0,0.0,1.0
147,6.5,3.0,5.2,2.0,0.0,0.0,1.0
148,6.2,3.4,5.4,2.3,0.0,0.0,1.0


In [41]:
# Prepare data for training with input rust functions expect
label_col = 2

# Normalize columns
# labels = encoded.iloc[:, label_col]
# max_per_col = encoded.abs().max().replace(0,1)
# encoded = (encoded / max_per_col)
# encoded.iloc[:, label_col] = labels

train_data, test_data = split_train_test(encoded, 0.9, True)

# Remove any encoded data of the same class
train_data_features = train_data.copy()
remove_encoded_category_data_and_label(train_data_features, feature_to_encoded_cols, label_col)

train_data_features_list = train_data_features.values.tolist()

# Data Correlation
corr = train_data_features.corr()
fig = px.imshow(corr, aspect="auto", origin='lower')
fig.show()

# Get labels
train_data_labels = train_data.iloc[:, label_col].values.tolist()
train_data_features

,sepal length (cm),sepal width (cm),petal width (cm),is_ class_Iris-setosa,is_ class_Iris-versicolor,is_ class_Iris-virginica
142,5.8,2.7,1.9,0.0,0.0,1.0
49,5.0,3.3,0.2,1.0,0.0,0.0
120,6.9,3.2,2.3,0.0,0.0,1.0
78,6.0,2.9,1.5,0.0,1.0,0.0
23,5.1,3.3,0.5,1.0,0.0,0.0
...,...,...,...,...,...,...
127,6.1,3.0,1.8,0.0,0.0,1.0
97,6.2,2.9,1.3,0.0,1.0,0.0
111,6.4,2.7,1.9,0.0,0.0,1.0
61,5.9,3.0,1.5,0.0,1.0,0.0


In [42]:
# Now train the model
hidden_layer_sizes = [20,20]
num_classifications = 1 # Tells the library to switch to regression mode
learning_rate = 1e-4
num_iterations = 20000
batch_size = 0
train_data_features_list
trained_model = rust_machine_learning.BasicNeuralNetwork(train_data_features_list, train_data_labels, hidden_layer_sizes, num_classifications, learning_rate, num_iterations, batch_size)

In [43]:
# Now let us test the model
test_data_features = test_data.copy()
remove_encoded_category_data_and_label(test_data_features, feature_to_encoded_cols, label_col)
test_data_features_list = test_data_features.values.tolist()
test_data_labels = test_data[test_data.columns[label_col]].values.tolist()

labelled_results = trained_model.regression(test_data_features_list)

num_correct = 0
tolerance = 0.05
for test_label, result_label in zip(test_data_labels, labelled_results):
  num_correct += (abs(test_label-result_label)/(1.0 if test_label == 0 else test_label) <= tolerance)
print("Percent Correct: ", 100.0 * num_correct / len(test_data_labels))
labelled_results

Percent Correct:  46.666666666666664


[1.496297883858824,
 3.877347283118251,
 1.3021182609007507,
 1.5864817738551824,
 5.015113973969816,
 1.4177594999074403,
 5.113908309066892,
 4.9124365430218315,
 5.315821184687016,
 1.6032063819525648,
 6.101582736402158,
 4.379791511122361,
 3.786981220543151,
 4.7359802891832,
 4.618242747227194]